In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import plotly.express as px

In [2]:
def pre_csv(df):
    df.dtm = pd.to_datetime(df.dtm)
    return df

def pre_dwd(df):
    df = df.to_dataframe().reset_index().rename(columns={"ref_datetime":"reference_time", "valid_datetime":"valid_time"})
    df.reference_time = df.reference_time.dt.tz_localize("UTC")
    df.valid_time = df.reference_time + df.valid_time * pd.Timedelta(1, "h")
    return df

def pre_ncep(df):
    return df

dwd_pes10

In [3]:
df_pes_0 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20200920_20231027.nc"))
df_pes_1 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20231027_20240108.nc"))
df_pes_2 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20240108_20240129.nc"))
df_pes_3 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_pes10_20240129_20240519.nc"))
# 9831960 rows
df_pes = pd.concat([df_pes_0, df_pes_1, df_pes_2, df_pes_3]).sort_values(["reference_time", "valid_time"]).reset_index(drop=True)
del df_pes_0, df_pes_1, df_pes_2, df_pes_3

In [4]:
# 491319 rows
df_pes = df_pes.groupby(["reference_time", "valid_time"]).mean().reset_index().drop(columns=["point", "longitude", "latitude"])
# 1273203 rows
df_pes = df_pes.set_index("valid_time").groupby(["reference_time"]).resample("30min").interpolate("linear").drop(columns="reference_time").reset_index()

In [5]:
df_pes.head()

,reference_time,valid_time,CloudCover,SolarDownwardRadiation,Temperature
0,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,0.450405,0.0,13.646173
1,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,0.472211,0.0,13.658508
2,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,0.494018,0.0,13.670843
3,2020-09-20 00:00:00+00:00,2020-09-20 01:30:00+00:00,0.520214,0.0,13.732101
4,2020-09-20 00:00:00+00:00,2020-09-20 02:00:00+00:00,0.546410,0.0,13.793360


dwd_hornsea

In [6]:
df_hornsea_0 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20200920_20231027.nc"))
df_hornsea_1 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20231027_20240108.nc"))
df_hornsea_2 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20240108_20240129.nc"))
df_hornsea_3 = pre_dwd(xr.open_dataset("data/dwd_icon_eu_hornsea_1_20240129_20240519.nc"))
# 17697528 rows
df_hornsea = pd.concat([df_hornsea_0, df_hornsea_1, df_hornsea_2, df_hornsea_3]).sort_values(["reference_time", "valid_time"]).reset_index(drop=True)
del df_hornsea_0, df_hornsea_1, df_hornsea_2, df_hornsea_3

In [7]:
# 491319 rows
df_hornsea = df_hornsea.groupby(["reference_time", "valid_time"]).mean().reset_index().drop(columns=["longitude", "latitude"])
# 1273203 rows
df_hornsea = df_hornsea.set_index("valid_time").groupby(["reference_time"]).resample("30min").interpolate("linear").drop(columns="reference_time").reset_index()

In [8]:
df_hornsea.head()

,reference_time,valid_time,RelativeHumidity,Temperature,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100
0,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,85.213745,15.416670,61.588081,62.085178,10.043627,11.802604
1,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,85.012253,15.412510,61.203667,61.726974,9.905537,11.648819
2,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,84.810768,15.408349,60.819256,61.368774,9.767447,11.495033
3,2020-09-20 00:00:00+00:00,2020-09-20 01:30:00+00:00,84.357880,15.451218,60.511028,61.111038,9.631039,11.354128
4,2020-09-20 00:00:00+00:00,2020-09-20 02:00:00+00:00,83.904999,15.494086,60.202801,60.853306,9.494630,11.213223


energy.csv

In [9]:
df_0 = pre_csv(pd.read_csv("data/Energy_Data_20200920_20240118.csv"))
df_1 = pre_csv(pd.read_csv("data/Energy_Data_20240119_20240519.csv"))
# 64224 rows
df = pd.concat([df_0, df_1]).sort_values(["dtm"]).reset_index(drop=True)
del df_0, df_1

df["Wind_MWh_credit"] = 0.5*df["Wind_MW"] - df["boa_MWh"] # Umrechnen in MWh und Abzug von BOA (BOA ist die Drosselung, die schon in MW miteingerechnet ist. Es wird aber mehr Strom produziert und vergütet, dashalb muss BOA wieder draufgerechnet werde, BOA ist immer negativ)
df["Solar_MWh_credit"] = 0.5*df["Solar_MW"]
# Sicherstellen, dass es immer 30min Schritte sind?

In [10]:
df.head()

,dtm,MIP,Solar_MW,Solar_capacity_mwp,Solar_installedcapacity_mwp,Wind_MW,SS_Price,boa_MWh,DA_Price,Wind_MWh_credit,Solar_MWh_credit
0,2020-09-20 00:00:00+00:00,20.06,0.0,2130.537493,2228.208777,996.284,2.50,0.0,32.17,498.142,0.0
1,2020-09-20 00:30:00+00:00,19.77,0.0,2130.537493,2228.208777,957.576,15.00,0.0,32.17,478.788,0.0
2,2020-09-20 01:00:00+00:00,28.68,0.0,2130.537493,2228.208777,941.044,47.95,0.0,32.00,470.522,0.0
3,2020-09-20 01:30:00+00:00,28.97,0.0,2130.537493,2228.208777,964.366,29.13,0.0,32.00,482.183,0.0
4,2020-09-20 02:00:00+00:00,28.19,0.0,2130.537493,2228.208777,918.432,28.95,0.0,31.99,459.216,0.0


Merge Dataframes

In [11]:
# Prüfen ob hierbei etwas verloren geht!!
# 1273203 rows
df_full = pd.merge(df_pes, df_hornsea, on=["reference_time", "valid_time"])
# 1273203 rows
df_full = df_full.merge(df[["dtm", "Wind_MWh_credit", "Solar_MWh_credit"]], left_on="valid_time", right_on="dtm", how="left")

In [12]:
# 1273203 rows, 14 columns
df_full.head()

,reference_time,valid_time,CloudCover,SolarDownwardRadiation,Temperature_x,RelativeHumidity,Temperature_y,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,dtm,Wind_MWh_credit,Solar_MWh_credit
0,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,0.450405,0.0,13.646173,85.213745,15.416670,61.588081,62.085178,10.043627,11.802604,2020-09-20 00:00:00+00:00,498.142,0.0
1,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,0.472211,0.0,13.658508,85.012253,15.412510,61.203667,61.726974,9.905537,11.648819,2020-09-20 00:30:00+00:00,478.788,0.0
2,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,0.494018,0.0,13.670843,84.810768,15.408349,60.819256,61.368774,9.767447,11.495033,2020-09-20 01:00:00+00:00,470.522,0.0
3,2020-09-20 00:00:00+00:00,2020-09-20 01:30:00+00:00,0.520214,0.0,13.732101,84.357880,15.451218,60.511028,61.111038,9.631039,11.354128,2020-09-20 01:30:00+00:00,482.183,0.0
4,2020-09-20 00:00:00+00:00,2020-09-20 02:00:00+00:00,0.546410,0.0,13.793360,83.904999,15.494086,60.202801,60.853306,9.494630,11.213223,2020-09-20 02:00:00+00:00,459.216,0.0


Testdaten abspalten

In [13]:
df_train = df_full.loc[df_full.reference_time < "2023-05-20"]
df_test = df_full.loc[df_full.reference_time >= "2023-05-20"]
del df_full